In [1]:
import tifffile
import matplotlib.pyplot as plt
import pyvista as pv
import cv2 as cv
import numpy as np
import itertools
import scipy
import time
from sklearn.mixture import GaussianMixture as GMM
from scipy.ndimage import generic_filter


In [2]:
files = ['B51_bag02.tif' , 'B51_bag40.tif' , 'B51_bag80.tif' , 'B51_bag120.tif']
input_path = ''
file_number = 3
img = tifffile.imread(input_path+files [file_number])
print (img.shape)


(8721, 1008, 1008)


In [3]:
def filter_region_of_interest(vol,threshold = 30):
    mask = np.zeros_like(vol)
    for i in range (vol.shape[0]):
        
        #Threshholding 
        ret,thresh = cv.threshold(vol[i], threshold, 1, cv.THRESH_BINARY)

        # Finding contours for the thresholded image
        contours, hierarchy = cv.findContours(thresh, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
        # Concatenate all contours into a single array
        all_points = np.concatenate(contours)
        #contours = sorted(contours, key=lambda x: cv.contourArea(x), reverse=True)
        convexHull = cv.convexHull(all_points)
        cv.drawContours(mask[i], [convexHull], -1, 1, -1 )
    
    return mask
    


In [4]:
def get_neighbours (vol, window_box = (3, 3, 3)):
    
    from skimage.util import view_as_windows
    
    #get window properties of single pixel 27 features
    original_shape = vol.shape

    

    neighborhoods = view_as_windows(vol, window_box)
    

    return neighborhoods,original_shape

In [5]:
def get_training_features(vol,neighborhoods, window_box = 3*3*3):
    
    ice_area = filter_region_of_interest(vol)
    
    training_n_elements = len(ice_area[ice_area > 0])
    trainig_features = np.zeros((training_n_elements, window_box))
    
    # resizing the ice area to the size that the neighbours are available (dropping one leayer each dimenssion)
    if window_box == 3*3*3:
        ice_area = ice_area [1:ice_area.shape[0]-1, 1:ice_area.shape[1]-1, 1:ice_area.shape[2]-1 ]
    else:
        print ('croping part needs to be modified')
    
    trainig_features = neighborhoods [ice_area > 0]
    trainig_features = trainig_features.reshape((trainig_features.shape[0], -1))
                    
    return trainig_features

In [6]:
def train_GMM (trainig_features):
    
    trainig_features = trainig_features.astype('float32')
    trainig_features = cv.normalize(trainig_features, None, alpha=0, beta=1, norm_type=cv.NORM_MINMAX, dtype=cv.CV_32F)
    
    gmm_model = GMM(n_components=2, random_state=20, covariance_type='tied', init_params='kmeans').fit(trainig_features)
    ice_class = gmm_model.means_.round(3).tolist()
    ice_class_index = ice_class.index(max(ice_class))
    
    return gmm_model, ice_class_index


In [7]:
def predict_with_GMM (gmm_model, ice_class_index, neighborhoods):
    
    neighborhoods = neighborhoods.astype('float32')
    neighborhoods = cv.normalize(neighborhoods, None, alpha=0, beta=1, norm_type=cv.NORM_MINMAX, dtype=cv.CV_32F)
    #shape>> (n,n,n,3,3,3)
    
    segmented = np.zeros((neighborhoods.shape[0],neighborhoods.shape[1],neighborhoods.shape[2]))
    
    reshaped_ne = neighborhoods.reshape(neighborhoods.shape[0],neighborhoods.shape[1],neighborhoods.shape[2] , -1)
    #shape>> (n,n,n,27)
    
    reshaped_ne = reshaped_ne.reshape(reshaped_ne.shape[0]*reshaped_ne.shape[1]*reshaped_ne.shape[2],reshaped_ne.shape[3])
    #shape>> (n,27)
    
    
    predictions = gmm_model.predict(reshaped_ne)
    predictions = predictions.reshape((neighborhoods.shape[0], neighborhoods.shape[1], neighborhoods.shape[2]))

    segmented[predictions == ice_class_index] = 100
    segmented[predictions != ice_class_index] = 0
    segmented = segmented.astype('uint8')
    
    return segmented
    
    

In [8]:
t1 = time.time()
batch = 100

#starting layer should be more than 1
starting_layer = 75
ending_layer = 8267

n_layers = ending_layer - starting_layer
cycle = n_layers//batch

# creating a numpy array to concatinate the reuslts into it
segmented = np.zeros((1,img.shape[1]-2,img.shape[2]-2), dtype = 'uint8')

for i in range (cycle):
    print(round(i*100//cycle), end='\r')
    
    if i == cycle-1: # the last batch will take some extra images
        vol = img[starting_layer+(i*batch)-1:ending_layer]
    else:
        vol = img[starting_layer+(i*batch)-1 : starting_layer+((i+1)*batch)+1 ]

    
    neighbours, _ = get_neighbours(vol) #shape>> (n, 3*3*3)
    trainig_features = get_training_features(vol, neighbours) #shape>> (n, 3*3*3) pixels only in region of interest
    gmm_model, ice_class_index = train_GMM (trainig_features)
    prediction = predict_with_GMM(gmm_model, ice_class_index, neighbours) #shape>> (n, 1008, 1008)
    segmented = np.concatenate((segmented, prediction),0)
    
t2 = time.time()
print (round(t2-t1))

37537


In [9]:
tifffile.imwrite(input_path+ '//GMM_3D//' + files [file_number], segmented)
tifffile.imwrite(input_path+ '//GMM_3D//' + 'corresponding_img_' + files [file_number], img[starting_layer:ending_layer, 1:-1, 1:-1])